# Парсинг данных. Практика 3

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
from time import sleep
from tqdm import tqdm

## Задание

Дана ссылка на первую страницу [каталога книжных новинок](https://www.bgshop.ru/catalog/group?id=444&page=1&sort=1&instock=). Соберите информацию о книгах со всех страниц. 

In [2]:
urls = []

for i in range(1, 5):
    urls.append(f'https://www.biblio-globus.ru/catalog/group?id=444&page={i}&sort=1&instock=')
urls

['https://www.biblio-globus.ru/catalog/group?id=444&page=1&sort=1&instock=',
 'https://www.biblio-globus.ru/catalog/group?id=444&page=2&sort=1&instock=',
 'https://www.biblio-globus.ru/catalog/group?id=444&page=3&sort=1&instock=',
 'https://www.biblio-globus.ru/catalog/group?id=444&page=4&sort=1&instock=']

In [3]:
page = requests.get(urls[1])
page.encoding='utf-8'

In [4]:
soup = BeautifulSoup(page.text)
soup

<!DOCTYPE html>
<html prefix="og: http://ogp.me/ns#">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="fd9a658378927472" name="mailru-verification"/>
<title>Книжные новинки | Библио-Глобус</title>
<meta content="Еженедельное обновление новой литературы от крупных издательств. Будь в курсе самых популярных книжных новинок!" name="description"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<meta content="all,follow" name="robots"/>
<script src="https://www.google.com/recaptcha/api.js?render=6LeAtKsiAAAAAB2-39qRtUXN68LwgjmLeY5q8I33"></script>
<link href="/favicon.png" rel="icon" type="image/png"/>
<link href="/favicon.ico" rel="icon" type="image/x-icon"/>
<link href="/searchprovider.xml" rel="search" title="Biblio-Globus" type="application/opensearchdescription+xml"/>
<link href="https://static1.bgshop.ru" rel="dns-prefetch"/>
<link href="https://static2.bgshop.ru" rel="dns-prefetch"/>
<link href="https://fonts.g

In [5]:
product = soup.find_all('div', {'class':'product'})[0]
product

<div class="product">
<a class="img_link" href="/product/10992234"><img alt="" class="img-fluid" src="https://static1.bgshop.ru/imagehandler.ashx?fileName=10992234.jpg&amp;width=200"/></a>
<div class="text">
<div class="product-sm-used-placeholder"></div>
<div class="author"> </div>
<h3><a href="/product/10992234" id="p_title_10992234">Квест-гид по Санкт-Петербургу. Необычный путеводитель по центру любимого города</a></h3>
<div class="prices_item">
<div class="price_item_wrapper"><span class="price_item_title">Цена на сайте:</span> <div class="price_item_block"><s>559 ₽</s> <span class="price_item_new price_item_with_discount">503 ₽</span></div></div>
</div>
<p class="status im_status_title">
<span style="color: #3ba155;"> в наличии</span>
<span class="product-qty">3 шт.</span>
</p>
<div class="buttons row">
<div class="col-9 pl-0">
<span class="btn btn-block btn-primary" id="add_10992234" onclick="AddToBasket(10992234)">В корзину</span>
<a class="btn btn-primary add_but" href="/Basket

In [10]:
'https://www.biblio-globus.ru' + product.find_all('a')[0].get('href')

'https://www.biblio-globus.ru/product/10992234'

In [12]:
product.find('div', class_="author").text

' '

In [11]:
product.find('h3').text

'Квест-гид по Санкт-Петербургу. Необычный путеводитель по центру любимого города'

In [13]:
price = product.find('span', class_="price_item_new price_item_with_discount").text
price

'503\xa0₽'

In [14]:
# сразу научимся чистить цену

price_clean = int(re.findall(r'\d+', price)[0])
price_clean

503

In [15]:
qty = soup.find('span', class_='product-qty').text
qty

'3 шт.'

In [16]:
qty_clean = int(re.findall(r'\d+', qty)[0])
qty_clean

3

Спарсим все данные.

In [52]:
books = []

for u in tqdm(urls):
    page = requests.get(u)
    soup = BeautifulSoup(page.text)
    sleep(1)
    info = soup.find_all('div', {'class':'product'})
    for i in info:
        author = i.find('div', class_="author").text
        title = i.find('h3').text
        try:
            price = i.find('span', class_="price_item_new price_item_with_discount").text            
        except:
            price = i.find('span', class_="price_item_new_lg").text # на случай, если цена без скидки
            
        price_clean = int(re.findall(r'\d+', price)[0])    
        qty = soup.find('span', class_='product-qty').text
        qty_clean = int(re.findall(r'\d+', qty)[0])
        url0 = 'https://www.bgshop.ru' + i.find('a').get('href')

        books.append((author, title, price_clean, qty_clean, url0))

100%|█████████████████████████████████████████████| 2/2 [00:04<00:00,  2.17s/it]


In [53]:
df = pd.DataFrame(books)
df.columns = ['Автор', 'Название', 'Цена', 'Количество', 'Ссылка']

In [54]:
df

Автор  \
0                                    А.Б. Широкорад   
1                                   Михалкова Е.И.    
2                                     Фернандес А.    
3                                          Т. Хайэм   
4                                      Ивлиева Ю.А.   
5                                        Лю Цысинь    
6                                         Лайелл Р.   
7                                      А.В Курпатов   
8                                        Чжунхо С.    
9                                         А. Феверс   
10                                         Дайо А.    
11                                   Васильев  А.А.   
12  Мединский В.Р., под науч. ред. Никифорова Ю.А.    
13                                       Лю Цысинь    
14                                    Лора Спенс-Эш   
15                                     Старгардт Н.   
16                                      Джонсон Д.    
17                                      О`Брайн П.    
18                                           Лю Д.    
19                                                    
20                                      Генис А.А.    
21                                               .    
22                                     Сэнсом К.Дж.   
23                                  Хабенский  К.Ю.   

                                             Название  Цена  Количество  \
0   Израиль - Палестина. 75 лет великого противост...   782          10   
1                       Нежные листья, ядовитые корни   611          10   
2                    Аргентина. Полная история страны   719          10   
3   Мир до нас: Новый взгляд на происхождение чело...   782          10   
4   Ты - причина. Почему мы всегда получаем то, че...   709          10   
5   Задача трех тел (Воспоминания о прошлом Земли ...   665          10   
6   Путешествия англичанина по России, Крыму, Кавк...  1554          10   
7                    Самоучитель 3 в 1. Полная версия  1988          10   
8                               Корея. Полная история   360          10   
9                                            Огневица   854          10   
10  Неизвестный Наполеон. Эпопея о величии и трагедии  1197          10   
11           Сокровища кочевника: Париж и далее везде  1197          10   
12  Рассказы из русской истории. Петр I. Начало. К...   566          16   
13                            О муравьях и динозаврах   890          16   
14                                      А дальше-море   951          16   
15          Свидетели войны. Жизнь детей при нацистах  1189          16   
16                                      Дымовое древо  1563          16   
17                    Большой исторический атлас мира  2098          16   
18                              Китай. Полная история   360          16   
19                                Шедевры архитектуры  1920          16   
20                           Трикотаж. Обратный адрес   908          16   
21                                Шедевры архитектуры  3111          16   
22                                           Доминион   942          16   
23  Московский художественный театр. МХТ 125 лет н...  1648          16   

                                    Ссылка  
0   https://www.bgshop.ru/product/10967171  
1   https://www.bgshop.ru/product/10967057  
2   https://www.bgshop.ru/product/10967017  
3   https://www.bgshop.ru/product/10966967  
4   https://www.bgshop.ru/product/10966889  
5   https://www.bgshop.ru/product/10966879  
6   https://www.bgshop.ru/product/10966806  
7   https://www.bgshop.ru/product/10966704  
8   https://www.bgshop.ru/product/10966534  
9   https://www.bgshop.ru/product/10966475  
10  https://www.bgshop.ru/product/10965329  
11  https://www.bgshop.ru/product/10964938  
12  https://www.bgshop.ru/product/10962288  
13  https://www.bgshop.ru/product/10961836  
14  https://www.bgshop.ru/product/10961754  
15  https://www.bgshop.ru/product/10961713  
16  https://www.bgshop.ru/pro

In [55]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Автор       24 non-null     object
 1   Название    24 non-null     object
 2   Цена        24 non-null     int64 
 3   Количество  24 non-null     int64 
 4   Ссылка      24 non-null     object
dtypes: int64(2), object(3)
memory usage: 1.1+ KB


In [56]:
df = df.sort_values('Цена', ascending = False).reset_index(drop=True)

In [57]:
df

Автор  \
0                                                .    
1                                       О`Брайн П.    
2                                      А.В Курпатов   
3                                                     
4                                   Хабенский  К.Ю.   
5                                       Джонсон Д.    
6                                         Лайелл Р.   
7                                          Дайо А.    
8                                    Васильев  А.А.   
9                                      Старгардт Н.   
10                                    Лора Спенс-Эш   
11                                     Сэнсом К.Дж.   
12                                      Генис А.А.    
13                                       Лю Цысинь    
14                                        А. Феверс   
15                                   А.Б. Широкорад   
16                                         Т. Хайэм   
17                                    Фернандес А.    
18                                     Ивлиева Ю.А.   
19                                       Лю Цысинь    
20                                  Михалкова Е.И.    
21  Мединский В.Р., под науч. ред. Никифорова Ю.А.    
22                                       Чжунхо С.    
23                                           Лю Д.    

                                             Название  Цена  Количество  \
0                                 Шедевры архитектуры  3111          16   
1                     Большой исторический атлас мира  2098          16   
2                    Самоучитель 3 в 1. Полная версия  1988          10   
3                                 Шедевры архитектуры  1920          16   
4   Московский художественный театр. МХТ 125 лет н...  1648          16   
5                                       Дымовое древо  1563          16   
6   Путешествия англичанина по России, Крыму, Кавк...  1554          10   
7   Неизвестный Наполеон. Эпопея о величии и трагедии  1197          10   
8            Сокровища кочевника: Париж и далее везде  1197          10   
9           Свидетели войны. Жизнь детей при нацистах  1189          16   
10                                      А дальше-море   951          16   
11                                           Доминион   942          16   
12                           Трикотаж. Обратный адрес   908          16   
13                            О муравьях и динозаврах   890          16   
14                                           Огневица   854          10   
15  Израиль - Палестина. 75 лет великого противост...   782          10   
16  Мир до нас: Новый взгляд на происхождение чело...   782          10   
17                   Аргентина. Полная история страны   719          10   
18  Ты - причина. Почему мы всегда получаем то, че...   709          10   
19  Задача трех тел (Воспоминания о прошлом Земли ...   665          10   
20                      Нежные листья, ядовитые корни   611          10   
21  Рассказы из русской истории. Петр I. Начало. К...   566          16   
22                              Корея. Полная история   360          10   
23                              Китай. Полная история   360          16   

                                    Ссылка  
0   https://www.bgshop.ru/product/10958326  
1   https://www.bgshop.ru/product/10961445  
2   https://www.bgshop.ru/product/10966704  
3   https://www.bgshop.ru/product/10960747  
4   https://www.bgshop.ru/product/10955592  
5   https://www.bgshop.ru/product/10961458  
6   https://www.bgshop.ru/product/10966806  
7   https://www.bgshop.ru/product/10965329  
8   https://www.bgshop.ru/product/10964938  
9   https://www.bgshop.ru/product/10961713  
10  https://www.bgshop.ru/product/10961754  
11  https://www.bgshop.ru/product/10956140  
12  https://www.bgshop.ru/product/10959118  
13  https://www.bgshop.ru/product/10961836  
14  https://www.bgshop.ru/product/10966475  
15  https://www.bgshop.ru/product/10967171  
16  https://www.bgshop.ru/pro

In [82]:
df.to_excel('bg_books.xlsx')

## Задание 2

По аналогии спарсите все книги, в которых есть слово "Python".

In [ ]:
# ваш код